# GPT-4 API: Extract Structured Data from SEC Filings (Material Event Disclosures)

Data sources used:
- SEC-API.io
- AlgoSeek
- WRDS Fundamental Data

Logic:

- Query API to download metadata of 8-Ks with Item 4.02 => create file `8K-filing-metadata.csv`
- Extractor API to extract and download Item 4.02 sections => save to `./data/edgar/8K-item-4.02/CIK/ACCESSION-NO-item-4-2.txt`
- Pre-select 100 filings that deal with revenue recognition errors, and where 50% impact is material and 50% not-material.
- For each item section, extract structured data using GPT4 and save to `./data/edgar/8K-item-4.02/CIK/ACCESSION-NO-item-4-2-structured-data.json`
- For each CIK, and `filedAt`, read `Years since IPO` and `market cap` from `fundamentals`
- For each filing, calculate 1/2/3/5/10/20 day return after filing was disclosed
  - Plot return distributions for each day
  - Display descriptive stats

More:
- https://sec-api.io/resources/analyze-8-k-filings-and-material-event-disclosure-activity

## Step 1: Locate URLs of 8-K Filings with Item 4.02

In [1]:
# load OPENAI_API_KEY value from .env file
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
!pip -q install sec-api


[notice] A new release of pip is available: 23.0 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [17]:
import os
from sec_api import QueryApi
import pandas as pd

SEC_API_KEY = os.getenv("SEC_API_KEY")

queryApi = QueryApi(api_key=SEC_API_KEY)

query = {
    "query": 'formType:"8-K" AND items:"4.02" AND filedAt:[2022-01-01 TO 2022-12-31]',
    "from": "0",
    "size": "50",
    "sort": [{"filedAt": {"order": "desc"}}],
}

response = queryApi.get_filings(query)

# Convert JSON array to DataFrame
filings_metadata = pd.DataFrame(response["filings"])

columns_of_interest = [
    "formType",
    "filedAt",
    "accessionNo",
    "ticker",
    "cik",
    "companyName",
    "items",
    "linkToHtml",
]

filings_metadata[columns_of_interest].head(10)

,formType,filedAt,accessionNo,ticker,cik,companyName,items,linkToHtml
0,8-K,2022-12-27T16:20:13-05:00,0001213900-22-082892,POLCQ,1810140,Polished.com Inc.,[Item 4.01: Changes in Registrant's Certifying...,https://www.sec.gov/Archives/edgar/data/181014...
1,8-K,2022-12-22T16:30:53-05:00,0001493152-22-036320,GLLI,1888734,GLOBALINK INVESTMENT INC.,[Item 4.02: Non-Reliance on Previously Issued ...,https://www.sec.gov/Archives/edgar/data/188873...
2,8-K,2022-12-21T17:25:38-05:00,0001213900-22-081850,CLRC,1903392,ClimateRock,[Item 4.02: Non-Reliance on Previously Issued ...,https://www.sec.gov/Archives/edgar/data/190339...
3,8-K,2022-12-16T16:17:36-05:00,0001185185-22-001426,FEIM,39020,FREQUENCY ELECTRONICS INC,[Item 2.02: Results of Operations and Financia...,https://www.sec.gov/Archives/edgar/data/39020/...
4,8-K,2022-12-16T16:05:27-05:00,0001493152-22-035733,VTRO,793171,"Vitro Biopharma, Inc.",[Item 4.02: Non-Reliance on Previously Issued ...,https://www.sec.gov/Archives/edgar/data/793171...
5,8-K,2022-12-13T17:03:52-05:00,0001437749-22-028985,BBCP,1703956,"Concrete Pumping Holdings, Inc.",[Item 4.02: Non-Reliance on Previously Issued ...,https://www.sec.gov/Archives/edgar/data/170395...
6,8-K,2022-12-13T16:05:01-05:00,0000950170-22-026457,ALCO,3545,"ALICO, INC.",[Item 2.02: Results of Operations and Financia...,https://www.sec.gov/Archives/edgar/data/3545/0...
7,8-K,2022-12-08T18:32:18-05:00,0001818502-22-000020,OPFI,1818502,OppFi Inc.,[Item 2.02: Results of Operations and Financia...,https://www.sec.gov/Archives/edgar/data/181850...
8,8-K,2022-12-08T16:01:02-05:00,0001213900-22-078469,GIAC,1853314,Gesher I Acquisition Corp.,[Item 4.02: Non-Reliance on Previously Issued ...,https://www.sec.gov/Archives/edgar/data/185331...
9,8-K,2022-12-06T16:25:39-05:00,0001171843-22-007863,WHLM,1013706,"Wilhelmina International, Inc.",[Item 4.02: Non-Reliance on Previously Issued ...,https://www.sec.gov/Archives/edgar/data/101370...


In [18]:
# Save the DataFrame to a CSV file ./data/edgar/8K-filing-metadata.csv
filings_metadata[columns_of_interest].to_csv("./data/edgar/8K-filing-metadata.csv", index=False)

## Step 2: Download Item 4.02 Sections

## Step 3: Use GPT-4 API to Extract Structured Data

## Step 4: Calculate X-Day Returns for each Filing

- Load daily stock prices for each company from `./data/historical-prices/CIK.csv`

## Step 5: Enrich Data with Fundamental Data

- Load market cap, years since IPO, sector, industry, and other fundamental data for each CIK from WRDS in `./data/fundamentals/fundamentals.csv`

## Step 6: Analyze and Visualize Data

## End Result

![Return Histogram](assets/returns-histogram-4.02.png)

![Descp Stats](assets/desc-stats-returns-4.02.png)